In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

import os

import h5py
import numpy as np
import seaborn as sns

from astropy import constants as const
from astropy import units as u

from injection_helpers import snapshot_to_energy_file

 
# Note: these units differ from the ones in `units.py`
# These match the code units (M_sun, pc, Myr), while `units.py` uses cgs

m_p = const.m_p.cgs.to(u.M_sun).value

cm = u.cm.to(u.pc)
g = u.g.to(u.M_sun)

initial_metallicity = 0.02 # gas-phase mas fraction

In [ ]:
run_name = "cluster_cooling_100"

inputs_dir  = os.path.join("../runs", run_name, "inputs")
outputs_dir = os.path.join("../runs", run_name, "outputs")

IC_filename = os.path.join(inputs_dir, "{}_ics.hdf5".format(run_name))

print(inputs_dir)
print(outputs_dir)

print(IC_filename)

In [ ]:
if "mhd" in run_name:
    with_MHD = True
else:
    with_MHD = False
    
print("with_MHD:         ", with_MHD)

In [ ]:
# initially I used single precision snapshot files, but that led to issues
# now I use double precision by default, but I've still left the capability 
# to create single precision initial conditions, by setting this flag to `False`
double_precision = True 
    
print("double_precision: ", double_precision)

# Create cluster initial conditions

In [ ]:
ic_file = h5py.File(IC_filename, "w")

In [ ]:
header_group = ic_file.create_group("Header")
particles_group = ic_file.create_group("PartType0")

In [ ]:
#use this multiplier to get the desired box size
# i.e. don't change the box_size line, but do change n_part_per_side (to change resolution)
multiplier = 1.5

box_size = int( multiplier*400 )
n_part_per_side = int( multiplier*100 )

dx = box_size / n_part_per_side

n_part_gas = n_part_per_side**3
n_part = np.array([n_part_gas, 0, 0, 0, 0, 0])

In [ ]:
header_group.attrs["NumPart_ThisFile"] = n_part
header_group.attrs["NumPart_Total"]    = n_part
header_group.attrs["NumPart_Total_HighWord"] = np.zeros(6)

header_group.attrs["MassTable"] = np.zeros(6)

header_group.attrs["Time"] = 0.
header_group.attrs["Redshift"] = 0.
header_group.attrs["BoxSize"] = float(box_size)
header_group.attrs["NumFilesPerSnapshot"] = 1
header_group.attrs["Omega0"] = 0.0
header_group.attrs["OmegaLambda"] = 0.0
header_group.attrs["HubbleParam"] = 1.0
header_group.attrs["Flag_Sfr"] = 0
header_group.attrs["Flag_Cooling"] = 1
header_group.attrs["Flag_StellarAge"] = 0
header_group.attrs["Flag_Metals"] = 1
header_group.attrs["Flag_Feedback"] = 0
if double_precision:
    header_group.attrs["Flag_DoublePrecision"] = 1
else:
    header_group.attrs["Flag_DoublePrecision"] = 0
header_group.attrs["Flag_IC_Info"] = 0




In [ ]:
if double_precision:
    float_size = 8
else:
    float_size = 4
    
print("float_size: ", float_size)

float_type = "<f{}".format(float_size)
int_type = "<u4"

particles_group.create_dataset("Coordinates",                (n_part_gas, 3), dtype=float_type)
particles_group.create_dataset("Density",                    (n_part_gas,  ), dtype=float_type)
particles_group.create_dataset("ElectronAbundance",          (n_part_gas,  ), dtype=float_type)
particles_group.create_dataset("InternalEnergy",             (n_part_gas,  ), dtype=float_type)
particles_group.create_dataset("Masses",                     (n_part_gas,  ), dtype=float_type)
particles_group.create_dataset("Metallicity",                (n_part_gas,  ), dtype=float_type)
particles_group.create_dataset("NeutralHydrogenAbundance",   (n_part_gas,  ), dtype=float_type)
particles_group.create_dataset("ParticleChildIDsNumber",     (n_part_gas,  ), dtype=int_type)
particles_group.create_dataset("ParticleIDGenerationNumber", (n_part_gas,  ), dtype=int_type)
particles_group.create_dataset("ParticleIDs",                (n_part_gas,  ), dtype=int_type)
particles_group.create_dataset("SmoothingLength",            (n_part_gas,  ), dtype=float_type)
particles_group.create_dataset("Velocities",                 (n_part_gas, 3), dtype=float_type)

if with_MHD:
    particles_group.create_dataset("MagneticField",          (n_part_gas, 3), dtype=float_type)

    # unused
    particles_group.create_dataset("DivergenceOfMagneticField",   (n_part_gas, ), dtype=float_type)
    particles_group.create_dataset("DivBcleaningFunctionPhi",     (n_part_gas, ), dtype=float_type)
    particles_group.create_dataset("DivBcleaningFunctionGradPhi", (n_part_gas, ), dtype=float_type)


In [ ]:
a = np.linspace(0,box_size,num=n_part_per_side+1)[1:]
x, y, z = np.meshgrid(a,a,a)

coords  = np.vstack([x.flatten(), y.flatten(), z.flatten()]).T
coords += dx * (np.random.random(size=coords.shape) - .5)*1e-3 
coords  = coords % box_size

particles_group["Coordinates"][...] = coords
del coords
del x
del y
del z

In [ ]:
# Calculate e_int

# mean molecular weight determined by initializing a snapshot with T=10**4
#  set in the params file, then looking at the InternalEnergy of snapshot_000
#  -- may not hold for other metallicities?
mu = 1.2194864
# T = 1e4
T = 344.626350734159 # equilibrium value (yt actually infers this to be 178.97956248958883 K)
gamma = 5/3

e_int = const.k_B * (T * u.Kelvin) / mu / const.m_p / (gamma-1)
e_int = e_int.to(u.pc**2 / u.Myr**2).value

In [ ]:
shape_1D = (n_part_gas, )
shape_2D = (n_part_gas, 3)

In [ ]:
density = 1.33 * m_p / cm**3
particles_group["Density"][...] = np.full(shape_1D, density, dtype=float_type)
print("Finished: Density")

particles_group["ParticleIDs"][...] = np.arange(1, n_part_gas+1, dtype=int_type)
print("Finished: ParticleIDs")

particles_group["Velocities"][...] = np.full(shape_2D, 0, dtype=float_type)
print("Finished: Velocities")

particles_group["InternalEnergy"][...] = np.full(shape_1D, e_int, dtype=float_type)
print("Finished: InternalEnergy")

particles_group["Masses"][...] = np.full(shape_1D, dx**3 * density, dtype=float_type)
print("Finished: Masses")

## I *think* the following are correct for making this a valid snapshot file
particles_group["ElectronAbundance"][...] = np.full(shape_1D, 1., dtype=float_type)
print("Finished: ElectronAbundance")

particles_group["Metallicity"][...] = np.full(shape_1D, initial_metallicity, dtype=float_type)
print("Finished: Metallicity")

particles_group["NeutralHydrogenAbundance"][...] = np.full(shape_1D, 0., dtype=float_type)
print("Finished: NeutralHydrogenAbundance")

particles_group["ParticleChildIDsNumber"][...] = np.full(shape_1D, 0, dtype=int_type)
print("Finished: ParticleChildIDsNumber")

particles_group["ParticleIDGenerationNumber"][...] = np.full(shape_1D, 0, dtype=int_type)
print("Finished: ParticleIDGenerationNumber")

smoothing_length = 5.25 * (box_size/400) / (n_part_per_side/150) # approximating from existing uniform snapshots
particles_group["SmoothingLength"][...] = np.full(shape_1D, smoothing_length, dtype=float_type)
print("Finished: SmoothingLength")


if with_MHD:
    particles_group["MagneticField"][...] = np.full(shape_2D, [0,0,5], dtype=float_type)
    print("Finished: MagneticField")

    # unused
    particles_group["DivergenceOfMagneticField"][...]   = np.full(shape_1D, 0., dtype=float_type)
    print("Finished: DivergenceOfMagneticField")

    particles_group["DivBcleaningFunctionPhi"][...]     = np.full(shape_1D, 0., dtype=float_type)
    print("Finished: DivBcleaningFunctionPhi")

    particles_group["DivBcleaningFunctionGradPhi"][...] = np.full(shape_1D, 0., dtype=float_type)
    print("Finished: DivBcleaningFunctionGradPhi")


In [ ]:
list(particles_group.keys())

In [ ]:
ic_file.close()

# Copy to outputs dir

In [ ]:
import shutil

In [ ]:
snapshot_000_filename = os.path.join(outputs_dir, "snapshot_000.hdf5")
shutil.copy2(IC_filename, snapshot_000_filename)

# Create initial energy file:

First, delete any existing energy file:

In [ ]:
energy_filename = os.path.join(outputs_dir, "energy.txt")

In [ ]:
%rm -f $energy_filename

Now create a new energy file

In [ ]:
snapshot_to_energy_file(snapshot_000_filename, energy_filename)

In [ ]:
%cat  $energy_filename

# What to do next?
 0) Make sure the params.base file is _actually_ using the correct box size.

 1) Add the first SN (`add_SN_to_snapshot.ipynb`)
 
 2) Create an initial params file (`create_new_params_file.ipynb`) 
 
 3) Start simulation using the respective `*_loop_test`